In [ ]:
import os
import sys
import urllib
import shutil
import time
import requests
from requests.auth import HTTPBasicAuth
from requests import Request, Session
import base64
import logging
import json
import pandas as pd
import numpy as np
import sqlalchemy as sqa
from pandas.io.json import json_normalize
import glob
from threading import Thread
import itertools
from functools import reduce

In [ ]:
ConnStr = "DRIVER = {Provide your driver name };TrustServerCertificate = yes;Authentication=ActiveDirectoryIntegrated;SERVER = GiveyourServernamewithport;DATABASE = DBNAME"

In [ ]:
PROXY_PASS =os.environ.get('PROXY_PASS')
proxies = {
    'http':'proxy address',
    'https':'proxy address'
}


In [ ]:
def ConnectToDB(ConnStr):
    params = urllib.parse.quote_plus(ConnStr)
    engine = sqa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    cnxn = engine.connect()
    cnxn.fast_executemany = True
    return cnxn

def LoadToSQL(tbl,cnxn,chunk):
    chunk.to_sql(name = tbl,con = cnxn,schema = 'dbo',if_exists = 'append',index = False, index_Label=None,chunksize = 100000)
    
def CreateTargetTable(tbl,cnxn,chunk):
    chunk.to_sql(name = tbl,con = cnxn;schema = 'dbo',if_exist = 'replace',index = False,index_Label = None,chunksize =100000,dtype  = None)

In [ ]:
def ParallelLoad(df,TargetTable,ConnString,func,ThreadLimit = 16):
    connections = []
    for i in range(0,ThreadLimit,1):
        connections.append(ConnectToDB(ConnString))
    threads=[]
    StepSize = int(df.shape[0]/ThreadLimit)
    for i in range(0,ThreadLimit):
        process = Thread(target=func,args = [TargetTable,connections[i],df[i*StepSize:(i*StepSize)+StepSize]])
        process.start()
        threads.append(process)
    for process in threads:
        process.join()
    threads = []
    if (StepSize*ThreadLimit)!= df.shape[0]:
        process = Thread(target=func,args = [TargetTable,connection[0],df(StepSize*ThreadLimit):df.shape[0]])
        process.start()
        threads.append(process)
    for process in threads:
        process.join()
    return

In [ ]:
def GetAccessToken(Key,Secret,Subscriber,Proxies):
    MyKey = os.environ.get('MyKey')
    MySecret = os.environ.get('MySecret')
    MyKey = 'ProvideYourKey'
    MySecret = 'ProvideYourSecret'
    
    s = Session()
    s.proxies.update(proxies)
    req = Request('POST','your api link',headers = {"content-type":"application/json"},data = '{"grant_type":"client_credentials"}',auth = (Key,Secret))
    prepped = req.prepare()
    resp = s.send(prepped)
    if(resp.status_code == 200):
        d = json.loads(resp.text)
        Access_token = d['access_token']
        Access_token = 'Bearer '+Access_token
    else:
        logging.warning("Unsucessful authorization. Please check the connection to API service.")
        sys.stdout.flush()
        Access_token ='Unable to authorize'
    return Access_token

In [ ]:
def GetAccessTokenDirect():
    MyKey = os.environ.get('MyKey')
    MySecret = os.environ.get('MySecret')
    
    MyKey = 'ProvideYourKey'
    MySecret = 'ProvideYourSecret'
    
    Access_token = GetAccessToken(MyKey,MySecret,proxies)
    return Access_token

In [ ]:
def GetAPIData(provideyourid):
    Access_token = GetAccessTokenDirect()
    TargetCnxn = ConnectToDB(ConnString)
    r = requests.get("provide your api link"+str(yourid),headers = {"Authorization":Access_token},proxies = proxies)
    df = pd.DataFrame()
    df['yourid'] = [str[yourid]]
    df['Data'] = [json.dumps(r.json())]
    df.to_sql(name = 'YourTableName',con = TargetCnxn, schema = 'dbo',if_exists ='append',index=False,index_label = None,chunksize = 100000)
    return

In [ ]:
GetAPIData(yourId)